In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Model
from keras.layers import Embedding, LSTM, Dense,Activation,concatenate,Input,Flatten,Reshape
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
rating = pd.read_csv('/content/gdrive/My Drive/Capstone/ratings.csv')
movies= pd.read_csv('/content/gdrive/My Drive/Capstone/movies.csv')

In [ ]:
users = rating['userId'].unique()
train_df = pd.DataFrame(columns= rating.columns)
test_df = pd.DataFrame(columns= rating.columns)
for u in users:
    userRows = rating[rating.userId==u]
    train, test = train_test_split(userRows, test_size=0.2, random_state=42, shuffle=True)
    train_df = train_df.append(train)
    test_df = test_df.append(test)

In [ ]:
X_train = train_df[['userId','movieId']]
y_train  = train_df['rating']
X_test = test_df[['userId','movieId']]
y_test  = test_df['rating']

In [ ]:
nmovies = rating['movieId'].max()
nusers = rating['userId'].max()

In [ ]:
users_input = Input(shape=(1,))
movies_input = Input(shape = (1,))

users_emb = Embedding(nusers+1, 20)(users_input)
movies_emb = Embedding(nmovies+1 , 20 )(movies_input)

users_reshaped = Reshape((20,))(users_emb)
movies_reshaped = Reshape((20,))(movies_emb)

user_movies = concatenate([users_reshaped, movies_reshaped], name='Concatenation')
output_layer = Dense(1 , activation = 'relu')(user_movies)

model = Model([users_input, movies_input], output_layer)

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['accuracy'])
model.fit(x = [X_train['userId'].array, X_train['movieId'].array], y =y_train , batch_size = 64 , epochs = 20 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
80419/80419 [==============================] - 57s 706us/step - loss: 2.8783 - accuracy: 0.2049
Epoch 2/20
80419/80419 [==============================] - 57s 707us/step - loss: 0.7515 - accuracy: 0.3183
Epoch 3/20
80419/80419 [==============================] - 57s 704us/step - loss: 0.7032 - accuracy: 0.3301
Epoch 4/20
80419/80419 [==============================] - 59s 737us/step - loss: 0.6867 - accuracy: 0.3348
Epoch 5/20
80419/80419 [==============================] - 58s 721us/step - loss: 0.6775 - accuracy: 0.3372
Epoch 6/20
80419/80419 [==============================] - 57s 710us/step - loss: 0.6707 - accuracy: 0.3399
Epoch 7/20
80419/80419 [==============================] - 57s 712us/step - loss: 0.6657 - accuracy: 0.3420
Epoch 8/20
80419/80419 [==============================] - 57s 707us/step - loss: 0.6622 - accuracy: 0.3428
Epoch 9/20
80419/80419 [==============================] - 57s 708us/step - loss: 0.6590 - accuracy: 0.3437
Epoch 10/20
80419/80419 [============

In [ ]:
score = model.evaluate([X_test['userId'].array, X_test['movieId'].array], y_test)
print(np.sqrt(score))

20417/20417 [==============================] - 0s 20us/step
[0.89214684 0.56127675]
